In [ ]:
def ClusterFormation(Features, FeaturesInEachCluester, NoOfFeatures):
    MyClusters =[]
    if(NoOfFeatures<3):
        NoOfFeatures = 3
    for i in range (NoOfFeatures):
        for j in range (i+1,NoOfFeatures):
            NewRow = []
            NewRow.append(Features[i])
            for k in range(FeaturesInEachCluester-1):
                NewRow.append(Features[j+k])
            MyClusters.append(NewRow)   
    return MyClusters

In [ ]:
print("Finding best performing Clusters")

clusterWiseBestAccuracy = pd.DataFrame(columns=['NoOfFeatures','Features','Accuracy','TimeInSec'])
for i in range(2,len(selectedColumns)):
    Clusters = ClusterFormation(selectedColumns,i,7-i)
    print("\n**************************************************************************************")
    print("No of Features: " , i)   
    AccuracyData = pd.DataFrame(columns=['Features','AdaBoost','Bagging','Sum','TimeInSec'])

    for CL in Clusters:
        STARTED = time.time()
        X_Train = None
        Y_Train = None
        X_Test = None
        Y_Test = None
        X_Train = pd.DataFrame(X_TrainMaster[CL]).copy()
        X_Test = pd.DataFrame(X_TestMaster[CL]).copy()
        Y_Train = pd.DataFrame(Y_TrainMaster).copy()
        Y_Test = pd.DataFrame(Y_TestMaster).copy()
        AdaBoost_Classifier = AdaBoostClassifier()
        AdaBoost_Classifier = AdaBoost_Classifier.fit(X_Train, Y_Train)
        AdaBoostAccuracy = metrics.accuracy_score(Y_Test, AdaBoost_Classifier.predict(X_Test))

        Bagging_Classifier = BaggingClassifier()
        Bagging_Classifier = Bagging_Classifier.fit(X_Train, Y_Train)
        BaggingAccuracy = metrics.accuracy_score(Y_Test, Bagging_Classifier.predict(X_Test))

        tot=(AdaBoostAccuracy+BaggingAccuracy)/2
        ENDED = time.time()
        timeTaken = int(ENDED - STARTED)
        newRow = {'Features':CL, 'AdaBoost':AdaBoostAccuracy, 'Bagging':BaggingAccuracy,'Sum':tot,'TimeInSec':timeTaken}
        AccuracyData = AccuracyData.append(newRow , ignore_index=True)
        print(newRow)

    tmp = pd.DataFrame(AccuracyData.sort_values('Sum',ascending=False)).copy()
    
    tmp = tmp.head(1)
    finalFeatures = tmp['Features'][tmp.index[0]]
    clusterWiseBestAccuracy = clusterWiseBestAccuracy.append({'NoOfFeatures':i,'Features':finalFeatures,
                'Accuracy':tmp['Sum'][tmp.index[0]], 'TimeInSec':tmp['TimeInSec'][tmp.index[0]]}, ignore_index=True)

    print(finalFeatures)
    
print("Best performing Clusters from each feature group:")
clusterWiseBestAccuracy

In [ ]:
print("Clusters sorted based on Accuracy:")

for i in range(len(clusterWiseBestAccuracy)-1):
    if(clusterWiseBestAccuracy['Accuracy'].values[i] >= clusterWiseBestAccuracy['Accuracy'].values[i+1]):
        clusterWiseBestAccuracy['Accuracy'].values[i+1] = clusterWiseBestAccuracy['Accuracy'].values[i]
        clusterWiseBestAccuracy['Features'].values[i+1] = clusterWiseBestAccuracy['Features'].values[i]
clusterWiseBestAccuracy 

In [ ]:
print("Clusters with unique Accuracy:")

clusterWiseBestAccuracy = clusterWiseBestAccuracy.drop_duplicates(subset='Accuracy')  
clusterWiseBestAccuracy

In [ ]:
print("Final Selected Clusters of Features :\n")

fastestFeatures = clusterWiseBestAccuracy['Features'].values[0]
clusterWiseBestAccuracy = clusterWiseBestAccuracy.sort_values('Accuracy',ascending=False)   
mostAccurateFeatures =  clusterWiseBestAccuracy['Features'].values[0]

print(fastestFeatures)
print(mostAccurateFeatures)

noOfModels=2
if(fastestFeatures==mostAccurateFeatures):
    noOfModels=1

FutIdentEND= time.time()